In [1]:
# Imports. Nothing to see here.

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import string

from imblearn.over_sampling import SMOTE
from gensim.models import Word2Vec
from collections import Counter

import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk import FreqDist, word_tokenize
from nltk.tokenize import RegexpTokenizer

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, plot_confusion_matrix, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB

import sqlite3

In [2]:
# databases within ['categories', 'podcasts', 'reviews', 'runs']

cnx = sqlite3.connect('database.sqlite')

podcast = pd.read_sql_query("SELECT * FROM podcasts", cnx)

categories = pd.read_sql_query("SELECT * FROM categories", cnx)

runs = pd.read_sql_query("SELECT * FROM runs", cnx)

reviews = pd.read_sql_query("SELECT * FROM reviews", cnx)

In [3]:
explore = pd.read_sql_query('''SELECT p.podcast_id, AVG(rating) 
                            FROM podcasts p 
                            JOIN categories c 
                            ON p.podcast_id = c.podcast_id 
                            JOIN reviews r 
                            ON p.podcast_id = r.podcast_id 
                            GROUP BY p.podcast_id
                            ORDER BY rating
                            ''', cnx)
explore


,podcast_id,AVG(rating)
0,a0004b1ef445af9dc84dad1e7821b1e3,1.000000
1,a00c300fce2e20fe832f5f5e6148987c,4.444444
2,a00cb09b7a0e02c88a2c26ab08236296,4.200000
3,a00d083d9ad46e4a014a9903c046d544,4.925170
4,a00e38f15aac723b046dcbffc21aca27,4.710526
...,...,...
46660,fffdfb5b49d0d47943e09f6213a346e5,5.000000
46661,fffe308414050768d3ce3782aa503b7d,5.000000
46662,fffe3f208a56dfecfaf6d0a7f8399d63,5.000000
46663,ffff66f98c1adfc8d0d6c41bb8facfd0,5.000000


In [4]:
data = pd.read_sql_query("SELECT * FROM podcasts p JOIN categories c ON p.podcast_id = c.podcast_id JOIN reviews r ON p.podcast_id = r.podcast_id", cnx)

In [5]:
data.drop(['itunes_url','itunes_id', 'created_at','title'], axis=1, inplace=True)

In [6]:
print(data.isna().sum())

podcast_id    0
slug          0
podcast_id    0
category      0
podcast_id    0
content       0
rating        0
dtype: int64


In [7]:
# Check for duplicated rows and preserve unique entries.

a = len(data)
data = data.drop_duplicates()
b = len(data)
print('# Number of duplicate rows dropped: {}'.format(a-b))

# Number of duplicate rows dropped: 2967


In [23]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1422198 entries, 0 to 1425164
Data columns (total 7 columns):
 #   Column      Non-Null Count    Dtype 
---  ------      --------------    ----- 
 0   podcast_id  1422198 non-null  object
 1   slug        1422198 non-null  object
 2   podcast_id  1422198 non-null  object
 3   category    1422198 non-null  object
 4   podcast_id  1422198 non-null  object
 5   content     1422198 non-null  object
 6   rating      1422198 non-null  int64 
dtypes: int64(1), object(6)
memory usage: 126.8+ MB


In [8]:
data.shape

(1422198, 7)

In [9]:
data['category'].unique()

array(['arts', 'arts-performing-arts', 'music', 'arts-design',
       'education', 'society-culture', 'arts-visual-arts', 'technology',
       'arts-food', 'society-culture-personal-journals', 'comedy',
       'arts-fashion-beauty', 'tv-film', 'society-culture-places-travel',
       'kids-family', 'religion-spirituality', 'business',
       'society-culture-philosophy', 'spirituality', 'business-careers',
       'christianity', 'hinduism', 'business-investing', 'judaism',
       'islam', 'buddhism'], dtype=object)

In [10]:
tokenizer = RegexpTokenizer(r'\w+')

stops = stopwords.words('english')
stops += list(string.punctuation)

In [11]:
data

,podcast_id,slug,podcast_id,category,podcast_id,content,rating
0,c61aa81c9b929a66f0c1db6cbe5d8548,backstage-at-tilles-center,c61aa81c9b929a66f0c1db6cbe5d8548,arts,c61aa81c9b929a66f0c1db6cbe5d8548,Thanks for providing these insights. Really e...,5
1,c61aa81c9b929a66f0c1db6cbe5d8548,backstage-at-tilles-center,c61aa81c9b929a66f0c1db6cbe5d8548,arts-performing-arts,c61aa81c9b929a66f0c1db6cbe5d8548,Thanks for providing these insights. Really e...,5
2,c61aa81c9b929a66f0c1db6cbe5d8548,backstage-at-tilles-center,c61aa81c9b929a66f0c1db6cbe5d8548,music,c61aa81c9b929a66f0c1db6cbe5d8548,Thanks for providing these insights. Really e...,5
3,c61aa81c9b929a66f0c1db6cbe5d8548,backstage-at-tilles-center,c61aa81c9b929a66f0c1db6cbe5d8548,arts,c61aa81c9b929a66f0c1db6cbe5d8548,Super excited to see this podcast grow. So man...,5
4,c61aa81c9b929a66f0c1db6cbe5d8548,backstage-at-tilles-center,c61aa81c9b929a66f0c1db6cbe5d8548,arts-performing-arts,c61aa81c9b929a66f0c1db6cbe5d8548,Super excited to see this podcast grow. So man...,5
...,...,...,...,...,...,...,...
1425160,ddd451a18055f0108edf79f8c3c9bf15,what-if-world-stories-for-kids,ddd451a18055f0108edf79f8c3c9bf15,kids-family,ddd451a18055f0108edf79f8c3c9bf15,I made this so all you guys that listen to thi...,5
1425161,a08a93f2ef58f847b263c84d96e891f5,inbox-besties-w-kate-doster-email-marketing-po...,a08a93f2ef58f847b263c84d96e891f5,business,a08a93f2ef58f847b263c84d96e891f5,I just discovered Kate Doster and the hype is ...,5
1425162,f9255ade54a8e12df362cde70b49dff9,the-trypod,f9255ade54a8e12df362cde70b49dff9,comedy,f9255ade54a8e12df362cde70b49dff9,This podcast as well as the “you can sit with ...,1
1425163,f9255ade54a8e12df362cde70b49dff9,the-trypod,f9255ade54a8e12df362cde70b49dff9,comedy,f9255ade54a8e12df362cde70b49dff9,Really crappy ep 2/4. I guess they can’t all b...,1


In [12]:
data ['slug'] = data['slug'].apply(lambda x: x.replace('-',' '))
data ['category'] = data['category'].apply(lambda x: x.replace('-',' '))
data ['category'] = data['category'].apply(lambda x: x.split()[0])
data

,podcast_id,slug,podcast_id,category,podcast_id,content,rating
0,c61aa81c9b929a66f0c1db6cbe5d8548,backstage at tilles center,c61aa81c9b929a66f0c1db6cbe5d8548,arts,c61aa81c9b929a66f0c1db6cbe5d8548,Thanks for providing these insights. Really e...,5
1,c61aa81c9b929a66f0c1db6cbe5d8548,backstage at tilles center,c61aa81c9b929a66f0c1db6cbe5d8548,arts,c61aa81c9b929a66f0c1db6cbe5d8548,Thanks for providing these insights. Really e...,5
2,c61aa81c9b929a66f0c1db6cbe5d8548,backstage at tilles center,c61aa81c9b929a66f0c1db6cbe5d8548,music,c61aa81c9b929a66f0c1db6cbe5d8548,Thanks for providing these insights. Really e...,5
3,c61aa81c9b929a66f0c1db6cbe5d8548,backstage at tilles center,c61aa81c9b929a66f0c1db6cbe5d8548,arts,c61aa81c9b929a66f0c1db6cbe5d8548,Super excited to see this podcast grow. So man...,5
4,c61aa81c9b929a66f0c1db6cbe5d8548,backstage at tilles center,c61aa81c9b929a66f0c1db6cbe5d8548,arts,c61aa81c9b929a66f0c1db6cbe5d8548,Super excited to see this podcast grow. So man...,5
...,...,...,...,...,...,...,...
1425160,ddd451a18055f0108edf79f8c3c9bf15,what if world stories for kids,ddd451a18055f0108edf79f8c3c9bf15,kids,ddd451a18055f0108edf79f8c3c9bf15,I made this so all you guys that listen to thi...,5
1425161,a08a93f2ef58f847b263c84d96e891f5,inbox besties w kate doster email marketing po...,a08a93f2ef58f847b263c84d96e891f5,business,a08a93f2ef58f847b263c84d96e891f5,I just discovered Kate Doster and the hype is ...,5
1425162,f9255ade54a8e12df362cde70b49dff9,the trypod,f9255ade54a8e12df362cde70b49dff9,comedy,f9255ade54a8e12df362cde70b49dff9,This podcast as well as the “you can sit with ...,1
1425163,f9255ade54a8e12df362cde70b49dff9,the trypod,f9255ade54a8e12df362cde70b49dff9,comedy,f9255ade54a8e12df362cde70b49dff9,Really crappy ep 2/4. I guess they can’t all b...,1


In [13]:
podcast['podcast_id'].nunique()

46665

In [14]:
############################ Visualize this ##############################

data['rating'].value_counts()

5    1265761
1      60381
4      41631
3      29006
2      25419
Name: rating, dtype: int64